In [1]:
import pandas as pd
import numpy as np





(6543, 3232)

In [90]:
df = pd.read_csv('./data/Most-Recent-Cohorts-Institution.csv', low_memory=False)

In [91]:
df = df.replace('PrivacySuppressed', np.nan)

In [92]:
df.shape

(6543, 3232)

In [93]:
nan_percentage = df.isna().mean()

# Filter out columns with more than 30% NaN values
df_cleaned = df.loc[:, nan_percentage <= 0.3].copy()


In [94]:
df_cleaned.shape

(6543, 689)

In [95]:
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('float64')]))
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('int64')]))
print(len(df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]))

363
15
311


In [96]:
err_cols = set()
for col in df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]:
    try:
        pd.to_numeric(df_cleaned[col], errors='raise')
    except:
        err_cols.add(col)


In [97]:
for column in df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]:
    if column not in err_cols:
        df_cleaned.loc[:, column] = pd.to_numeric(df_cleaned[column], errors='coerce')


In [101]:
df_cleaned.columns[df_cleaned.dtypes.values == np.dtype('object')]

Index(['INSTNM', 'CITY', 'STABBR', 'ZIP', 'ACCREDAGENCY', 'INSTURL', 'NPCURL',
       'SEPAR_DT_MDN', 'T4APPROVALDATE', 'ACCREDCODE', 'BBRR1_FED_UG_DFLT',
       'BBRR1_FED_UG_DLNQ', 'BBRR1_FED_UG_FBR', 'BBRR1_FED_UG_DFR',
       'BBRR1_FED_UG_NOPROG', 'BBRR1_FED_UG_MAKEPROG',
       'BBRR1_FED_UG_PAIDINFULL', 'BBRR1_FED_UG_DISCHARGE',
       'BBRR2_FED_UG_DFLT', 'BBRR2_FED_UG_DLNQ', 'BBRR2_FED_UG_FBR',
       'BBRR2_FED_UG_DFR', 'BBRR2_FED_UG_NOPROG', 'BBRR2_FED_UG_MAKEPROG',
       'BBRR2_FED_UG_PAIDINFULL', 'BBRR2_FED_UG_DISCHARGE',
       'BBRR2_FED_UGCOMP_FBR', 'BBRR2_FED_UGCOMP_NOPROG', 'FEDSCHCD',
       'BBRR3_FED_UG_DFLT', 'BBRR3_FED_UG_DLNQ', 'BBRR3_FED_UG_FBR',
       'BBRR3_FED_UG_DFR', 'BBRR3_FED_UG_NOPROG', 'BBRR3_FED_UG_MAKEPROG',
       'BBRR3_FED_UG_PAIDINFULL', 'BBRR3_FED_UG_DISCHARGE',
       'BBRR3_FED_UGCOMP_FBR', 'BBRR3_FED_UGCOMP_DFR',
       'BBRR3_FED_UGCOMP_NOPROG', 'BBRR3_FED_UGCOMP_MAKEPROG',
       'BBRR3_FED_UGCOMP_PAIDINFULL', 'BBRR4_FED_UG_DFLT', 'BBRR4_

In [108]:
# Borrower-Based Repayment Rate on Federal Loans: 
# BBRR[YR]_[LOAN]_[GROUP]_[STATUS]
# [YR]=1, 2, 3, or 4
# [STATUS]:
# DFTL - Default - Not paid for 360+ days
# DLNQ - Delinquent - Not paying for 31 to 360 days
# FBR - Forbearance - Temporary stop
# DFR - Deferment 
# NOPROG - Not making progress
# MAKEPROG - Making progress
# PAIDINFULL - Paid in full
# Discharged - Discharged - obligation to pay removed

# questionable_cols = ['BBRR1_FED_UG_DFLT', 'BBRR1_FED_UG_DLNQ', '']
# val_cat_cols = ['ACCREDAGENCY', 'ACCREDCODE']
# dates = ['SEPAR_DT_MDN', 'T4APPROVALDATE']

df_cleaned['BBRR1_FED_UG_DFLT'].value_counts()

0.00      1423
<=0.02     936
<=0.05     874
<=0.01     787
<=0.10     520
<=0.20     421
0.01       117
Name: BBRR1_FED_UG_DFLT, dtype: int64